This notebook implements the users flows described in the following [page](https://www.figma.com/file/Khdaki93teaflfVDbVYM7F/Interfacer-User-Flows) .

In [ ]:
# The following should reload if_lib if we change it on disk
%load_ext autoreload
%aimport if_lib, if_utils, if_dpp, if_graphics, if_consts
%autoreload 1
import os
import json
import random

from if_utils import get_filename, show_data, save_traces

from if_lib import generate_random_challenge, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource_spec_id, get_resource, get_process, create_event, make_transfer, reduce_resource, set_user_location

from if_dpp import trace_query, check_traces, er_before, get_dpp

from if_graphics import vis_dpp, make_sankey, consol_trace

In [ ]:
# We define the constant for our case
USE_CASE = 'ifusersflows'

# What endpoint are we talking to?
# debug
# ENDPOINT = 'http://65.109.11.42:10000/api'
ENDPOINT = 'http://zenflows-debug.interfacer.dyne.org/api'
# staging
# ENDPOINT = 'http://65.109.11.42:8000/api'
# ENDPOINT = 'https://zenflows-staging.interfacer.dyne.org/api'
# testing
# ENDPOINT = 'http://65.109.11.42:9000/api'
# ENDPOINT = 'https://zenflows-test.interfacer.dyne.org/api'
USERS = ['designer1', 'designer2', 'service_prov1', 'service_prov2', 'manufacturer1', 'manufacturer2', 'customer1', 'customer2']

In [ ]:
# Calculate names of settings files
USERS_FILE = get_filename('cred_users.json', ENDPOINT, USE_CASE)
LOCS_FILE = get_filename('loc_users.json', ENDPOINT, USE_CASE)
UNITS_FILE = get_filename('units_data.json', ENDPOINT, USE_CASE)
SPECS_FILE = get_filename('res_spec_data.json', ENDPOINT, USE_CASE)

In [ ]:
# Read or define user data that is going to be used in the GraphQL calls

# create data structure to hold processes
process_data = {}

# create data structures to hold resources and events (possibly to compare results from track and trace)
res_data = {}
event_seq = []

if os.path.isfile(USERS_FILE):
    with open(USERS_FILE,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['designer1'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "Designer1",
      "username": "designer1_username",
      "email": "designer1@example.org",
      "note": "me.designer1.org"
    }
    users_data['designer2'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fido",
        "nameFirstTeacher": "Mary",
        "whereHomeTown": "Amsterdam",
        "nameMotherMaid": "Wraight"
      },
      "name": "Designer2",
      "username": "designer2_username",
      "email": "designer2@example.org",
      "note": "me.designer2.org"
    }

    users_data['manufacturer1'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "Manufacturer1",
        "username": "manufacturer1_username",
        "email": "manufacturer1@example.org",
        "note" : "me.manufacturer1.org"
    }
    users_data['manufacturer2'] = {
        "userChallenges": {
            "whereParentsMet":"Canicatti",
            "nameFirstPet":"Ula",
            "nameFirstTeacher":"Pugsly",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Ranks"
        },
        "name": "Manufacturer2",
        "username": "manufacturer2_username",
        "email": "manufacturer2@example.org",
        "note" : "me.manufacturer2.org"
    }
    users_data['service_prov1'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "Service_prov1",
        "username": "service_prov1_username",
        "email": "service_prov1@example.org",
        "note" : "me.service_prov1.com"
    }
    users_data['service_prov2'] = {
        "userChallenges": {
            "whereParentsMet":"Canicatti",
            "nameFirstPet":"Ula",
            "nameFirstTeacher":"Pugsly",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Ranks"
        },
        "name": "service_prov2",
        "username": "service_prov2_username",
        "email": "service_prov2@example.org",
        "note" : "me.service_prov2.com"
    }


    users_data['customer1'] = {
        "userChallenges": {
            "whereParentsMet":"Rome",
            "nameFirstPet":"Ku",
            "nameFirstTeacher":"George",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Canti"
        },
        "name": "Customer1",
        "username": "customer1_username",
        "email": "customer1@example.org",
        "note" : "me.customer1.org"
    }

    users_data['customer2'] = {
        "userChallenges": {
            "whereParentsMet":"Rome",
            "nameFirstPet":"Ku",
            "nameFirstTeacher":"George",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Canti"
        },
        "name": "Customer2",
        "username": "customer2_username",
        "email": "customer2@example.org",
        "note" : "me.customer2.org"
    }

    with open(USERS_FILE,'w') as f:
        json.dump(users_data, f)

if os.path.isfile(LOCS_FILE):
    with open(LOCS_FILE,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['designer1'] = {
        "name": "Dyne",
        "lat": 52.39679,
        "long": 4.8781073,
        "addr": "Haparandadam 7, A1, 1013 AK Amsterdam, Netherlands",
        "note": "location.designer1.org"
    }
    locs_data['designer2'] = {
        "name": "Farback",
        "lat": 52.3767127,
        "long": 4.8990591,
        "addr": "Prins Hendrikkade 82 A, 1012 AE, Amsterdam, Netherlands",
        "note": "location.designer2.org"
    }

    locs_data['manufacturer1'] = {
        "name": "Fab Lab Hamburg",
        "lat" : 51.1531305,
        "long" : 5.2685045,
        "addr" : "Stockmeyerstr. 43 Halle 4K Eingang von der Wasserseite, 20457 Hamburg, Germany",
        "note": "location.manufacturer1.org"
    }

    locs_data['manufacturer2'] = {
        "name": "Fab Lab Amsterdam",
        "lat" : 52.372773,
        "long" : 4.8981243,
        "addr" : "Nieuwmarkt 4, 1012 CR Amsterdam, Netherlands",
        "note": "location.manufacturer2.org"
    }

    locs_data['service_prov1'] = {
        "name": "Bike Totaal B.V.",
        "lat" : 52.1971579,
        "long" : 5.3924199,
        "addr" : "Spaceshuttle 22, 3824 ML Amersfoort, Netherlands",
        "note": "location.service_prov1.com"
    }

    locs_data['service_prov2'] = {
        "name": "Lyondell Covestro Manufacturing",
        "lat" : 51.9650382,
        "long" : 4.0133683,
        "addr" : "Australiëweg 7, 3199 KB Maasvlakte Rotterdam, Netherlands",
        "note": "location.service_prov2.com"
    }

    locs_data['customer1'] = {
        "name": "CleanLease",
        "lat" : 51.47240440868687,
        "long" : 5.412460440524406,
        "addr" : "De schakel 30, 5651 Eindhoven, Netherlands",
        "note": "location.customer1.org"
    }

    locs_data['customer2'] = {
        "name": "OLVG",
        "lat" : 52.3585703,
        "long" : 4.9124307,
        "addr" : "Oosterpark 9, 1091 AC Amsterdam, Netherlands",
        "note": "location.customer2.org"
    }

    with open(LOCS_FILE,'w') as f:
        json.dump(locs_data, f)

if os.path.isfile(UNITS_FILE):
    with open(UNITS_FILE,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)


if os.path.isfile(SPECS_FILE):
    with open(SPECS_FILE,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}

In [ ]:
# Read HMAC or get it from the server
for user in USERS:
    read_HMAC(USERS_FILE, users_data, user, endpoint=ENDPOINT)

In [ ]:
# Read the keypair
for user in USERS:
    read_keypair(USERS_FILE, users_data, user)

In [ ]:
# read or get id of the person
for user in USERS:
    get_id_person(USERS_FILE, users_data, user, endpoint=ENDPOINT)

In [ ]:
# Read of get the location id
for user in USERS:
    get_location_id(LOCS_FILE, users_data[user], locs_data, user, endpoint=ENDPOINT)
    set_user_location(USERS_FILE, users_data, locs_data, user, endpoint=ENDPOINT)

In [ ]:
# Get the ids of all units
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'piece', 'u_piece', 'om2:one', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'mass', 'kg', 'om2:kilogram', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'volume', 'lt', 'om2:litre', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'time', 'h', 'om2:hour', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['designer1'], units_data, 'distance', 'km', 'om2:kilometer', endpoint=ENDPOINT)

In [ ]:
# Create the processes

# Create the process that delivers the service frame manufacturing
process_name = 'Create_bike_frame_manifacturing_service_ABC'
user_data = users_data['service_prov1']
note = f"Creation of bike frame manufacturing service ABC by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that delivers bike assembly service
process_name = 'Create_bike_assembly_service'
user_data = users_data['service_prov2']
note = f"Creation of bike assembly 1-hour service by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that deliver the service frame manufacturing
process_name = 'Create_aluminium_bike_frame'
user_data = users_data['manufacturer1']
note = f"Creation of aluminum bike frame by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps creating design
process_name = 'Create_funky_bike_design'
user_data = users_data['designer1']
note = f"Creation of funky bike design by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps creating the mirror
process_name = 'Creation_magic_bike_mirror'
user_data = users_data['manufacturer2']
note = f"Creation of magic bike mirror by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps creating the bike
process_name = 'Creation_fancy_collaborative_bike'
user_data = users_data['designer2']
note = f"Creation of fancy collaborative bike by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)


In [ ]:
# Read all the resource specifications
name = 'bike_manufacturing_service'
note = 'Specification bike assembly service'
classification = 'https://www.service_prov1.com/manufacture/bike'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['service_prov1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'aluminium'
note = 'Aluminium 10mt, 4cm pipes'
classification = 'https://www.wikidata.org/wiki/Q663'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'aluminium_bike_frame'
note = 'Repository for aluminium bike frame'
classification = 'https://github.com/manufacturer1/aluminium_bike_frame'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'funky_bike_design'
note = 'Repository for funky bike design'
classification = 'https://github.com/designer1/funky_bike_design'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['designer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'glass'
note = 'Glass for magic bike mirror'
classification = 'https://www.wikidata.org/wiki/Q11469'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'magic_bike_mirror'
note = 'Repository for magic bike mirror'
classification = 'https://github.com/manufacturer2/magic_bike_mirror'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['manufacturer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'bike_assembly_service'
note = 'Specification bike assembly service'
classification = 'https://www.service_prov2.com/assembly/bike'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['service_prov2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'fancy_collaborative_bike'
note = 'Repository for fancy collaborative bike'
classification = 'https://github.com/designer2/fancy_collaborative_bike'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['designer2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'manufacturing_work'
note = 'Specification for manufacturing work'
classification = 'https://www.wikidata.org/wiki/Q187939'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['service_prov1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'assemblying_work'
note = 'Specification for assemblying work'
classification = 'https://www.wikidata.org/wiki/Q187939'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['service_prov2'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'design_work'
note = 'Specification for design work'
classification = 'https://www.wikidata.org/wiki/Q82604'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['designer1'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)




In [ ]:
# We create the resources that will not be saved to file as it is assumed they are recreated at each run

res_name = 'aluminium'
amount = 5
get_resource(res_data, res_spec_data, res_name, users_data['manufacturer1'], event_seq, amount, endpoint=ENDPOINT)

res_name = 'glass'
amount = .2
get_resource(res_data, res_spec_data, res_name, users_data['manufacturer2'], event_seq, amount, endpoint=ENDPOINT)



In [ ]:
# Define event work for the service creation
cur_res = action = event_note = amount = cur_pros = None
action = 'work'
event_note='work to manufacture bike frame'

cur_pros = process_data['Create_bike_frame_manifacturing_service_ABC']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['manufacturing_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['manufacturing_work']['id']
effort_spec['amount'] = 8


event_id, ts = create_event(users_data['service_prov1'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})

In [ ]:
# Define event produce for manufacturing the frame
cur_res = action = event_note = amount = cur_pros = None
action = 'deliverService'
event_note='produce service manufacturing the frame'
amount = 1

cur_outpros = process_data['Create_bike_frame_manifacturing_service_ABC']
cur_inpros = process_data['Create_aluminium_bike_frame']


cur_res = {
    "spec_id": res_spec_data['bike_manufacturing_service']['id']
}

event_id, ts = create_event(users_data['service_prov1'], action, event_note, amount=amount, process=cur_inpros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT, process2=cur_outpros)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action})


In [ ]:
# # use service manufacturing the frame as input
# cur_res = action = event_note = amount = cur_pros = None
# action = 'deliverService'
# event_note='produce service manufacturing the frame'
# amount = 1
# cur_pros = process_data['Create_aluminium_bike_frame']

# cur_res = {
#     "spec_id": res_spec_data['bike_manufacturing_service']['id']
# }

# event_id, ts = create_event(users_data['manufacturer1'], action, event_note, amount=amount, process=cur_pros, \
#                  res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

# event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action})


In [ ]:
# consume aluminium for bike frame
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume aluminium for bike frame'
amount = 5
cur_pros = process_data['Create_aluminium_bike_frame']
cur_res = res_data['aluminium_res']


event_id, ts = create_event(users_data['manufacturer1'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# Define event produce for aluminium bike frame
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce aluminium bike frame'
amount = 1
cur_pros = process_data['Create_aluminium_bike_frame']

res_data['aluminium_bike_frame_res'] = {
    "res_ref_id": f'aluminium_bike_frame-{random.randint(0, 10000)}',
    "name": 'aluminium bike frame',
    "spec_id": res_spec_data['aluminium_bike_frame']['id']
}
cur_res = res_data['aluminium_bike_frame_res']


event_id, ts = create_event(users_data['manufacturer1'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the aluminium bike frame from manufacturer1 to designer2
cur_res = action = event_note = amount = cur_pros = None
note='Transfer aluminium bike frame from manufacturer1 to designer2'
action = 'transfer'
amount = 1
cur_res = res_data['aluminium_bike_frame_res']

event_id, ts = make_transfer(users_data['manufacturer1'], action, note, users_data['designer2'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume glass for mirror
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume glass for bike mirror'
amount = .2
cur_pros = process_data['Creation_magic_bike_mirror']
cur_res = res_data['glass_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# Define event produce for aluminium bike frame
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce magic bike mirror'
amount = 1
cur_pros = process_data['Creation_magic_bike_mirror']

res_data['magic_bike_mirror_res'] = {
    "res_ref_id": f'magic_bike_mirror-{random.randint(0, 10000)}',
    "name": 'magic bike mirror',
    "spec_id": res_spec_data['magic_bike_mirror']['id']
}
cur_res = res_data['magic_bike_mirror_res']


event_id, ts = create_event(users_data['manufacturer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the mirror from manufacturer2 to designer2
cur_res = action = event_note = amount = cur_pros = None
note='Transfer magic bike mirror from manufacturer2 to designer2'
action = 'transfer'
amount = 1
cur_res = res_data['magic_bike_mirror_res']

event_id, ts = make_transfer(users_data['manufacturer2'], action, note, users_data['designer2'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Define event work for the service creation
cur_res = action = event_note = amount = cur_pros = None
action = 'work'
event_note='work to create funky bike design'

cur_pros = process_data['Create_funky_bike_design']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['design_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['design_work']['id']
effort_spec['amount'] = 16


event_id, ts = create_event(users_data['designer1'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})

In [ ]:
# Define event produce for aluminium bike frame
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce design for funky bike'
amount = 1
cur_pros = process_data['Create_funky_bike_design']

res_data['funky_bike_design_res'] = {
    "res_ref_id": f'funky_bike_design-{random.randint(0, 10000)}',
    "name": 'funky bike design',
    "spec_id": res_spec_data['funky_bike_design']['id']
}
cur_res = res_data['funky_bike_design_res']


event_id, ts = create_event(users_data['designer1'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Define event work for the service creation
cur_res = action = event_note = amount = cur_pros = None
action = 'work'
event_note='work to assemble bike'

cur_pros = process_data['Create_bike_assembly_service']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['manufacturing_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['manufacturing_work']['id']
effort_spec['amount'] = 4


event_id, ts = create_event(users_data['service_prov2'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})

In [ ]:
# Define event that deliver the service of bike assemblying
cur_res = action = event_note = amount = cur_pros = None
action = 'deliverService'
event_note='produce service for bike assemblying'
amount = 1

cur_outpros = process_data['Create_bike_assembly_service']
cur_inpros = process_data['Creation_fancy_collaborative_bike']


cur_res = {
    "spec_id": res_spec_data['bike_assembly_service']['id']
}

event_id, ts = create_event(users_data['manufacturer1'], action, event_note, amount=amount, process=cur_inpros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT, process2=cur_outpros)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action})


In [ ]:
# # Define event that deliver the service of bike assemblying
# cur_res = action = event_note = amount = cur_pros = None
# action = 'deliverService'
# event_note='use service for bike assemblying'
# amount = 1
# cur_pros = process_data['Creation_fancy_collaborative_bike']

# cur_res = {
#     "spec_id": res_spec_data['bike_assembly_service']['id']
# }

# event_id, ts = create_event(users_data['manufacturer1'], action, event_note, amount=amount, process=cur_pros, \
#                  res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

# event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action})


In [ ]:
# consume mirror for bike
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume mirror for bike'
amount = 1
cur_pros = process_data['Creation_fancy_collaborative_bike']
cur_res = res_data['magic_bike_mirror_res']


event_id, ts = create_event(users_data['designer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# cite design for bike
cur_res = action = event_note = amount = cur_pros = None
action = 'cite'
event_note='cite design to build bike'
amount = 1
cur_pros = process_data['Creation_fancy_collaborative_bike']
cur_res = res_data['funky_bike_design_res']


event_id, ts = create_event(users_data['designer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume aluminium bike frame for bike
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume aluminium bike frame'
amount = 1
cur_pros = process_data['Creation_fancy_collaborative_bike']
cur_res = res_data['aluminium_bike_frame_res']


event_id, ts = create_event(users_data['designer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# Define event produce for aluminium bike frame
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce fancy collaborative bike'
amount = 1
cur_pros = process_data['Creation_fancy_collaborative_bike']

res_data['fancy_collaborative_bike_res'] = {
    "res_ref_id": f'fancy_collaborative_bike-{random.randint(0, 10000)}',
    "name": 'fancy collaborative bike',
    "spec_id": res_spec_data['fancy_collaborative_bike']['id']
}
cur_res = res_data['fancy_collaborative_bike_res']


event_id, ts = create_event(users_data['designer2'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
show_data(users_data, locs_data, res_data, units_data, res_spec_data, process_data, event_seq)

In [ ]:
trace_me = res_data['fancy_collaborative_bike_res']['id']
print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['designer2'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

print(json_object)
print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, users_data['designer2'], endpoint=ENDPOINT)
print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['designer2'], endpoint=ENDPOINT)
check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(USE_CASE, tot_dpp, trace, be_dpp, event_seq)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)